In [123]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [124]:
df=pd.read_csv("./mnist.csv")

In [125]:
feature = df[df.columns[:-1]]
target = df[df.columns[-1]]

In [126]:
x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=1/7, random_state=42, stratify=target)

In [127]:
x_train = torch.tensor(x_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
x_test = torch.tensor(x_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.long)

In [128]:
# 데이터셋 생성
import torch.utils.data.dataloader as dataloader
import torch.utils.data.dataset as dataset

class MyDataset(dataset.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [129]:
dataset_train = MyDataset(x_train, y_train)
dataset_test = MyDataset(x_test, y_test)

In [130]:
class model(nn.Module): # 가중치 초기화 
    def __init__(self):
        super(model, self).__init__()
        self.fc1 = nn.Linear(784, 300)
        self.fc2 = nn.Linear(300, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [131]:
model2 = model()
optimizer = optim.Adam(model2.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

train_loader = dataloader.DataLoader(dataset_train, batch_size=100, shuffle=True)
test_loader = dataloader.DataLoader(dataset_test, batch_size=100, shuffle=False)

In [132]:
ex=torch.tensor([1])
ex.numpy()[0]

1

In [133]:
from torchmetrics.functional.classification import accuracy, f1_score
model2.train()

for epoch in range(15):
    
    for x,y in train_loader:
        output = model2(x)
        loss = criterion(output, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            #print(f"loss: {loss.item()}")
            pass
        
        
        

    print(f"================================={epoch}=================================")
    output = model2(x_train)
    acc = accuracy(output, y_train, task="multiclass", num_classes=10, average="macro")
    f1 = f1_score(output, y_train, task="multiclass", num_classes=10, average="macro")
    print(f"acc : {acc}, f1 : {f1}")
    
    
    output = model2(x_test)
    print(f"=================================test score=================================")
    print(accuracy(output, y_test, task="multiclass", num_classes=10, average="macro"))
    print(f1_score(output, y_test, task="multiclass", num_classes=10, average="macro"))

    if np.abs(f1-1) < 0.000001:
        print("발동!")
        torch.save(model2,"./model.pth")
        break

=================================0=================================
acc : 0.9719768762588501, f1 : 0.9718669652938843
=================================test score=================================
tensor(0.9651)
tensor(0.9650)
=================================1=================================
acc : 0.973682165145874, f1 : 0.9737364053726196
=================================test score=================================
tensor(0.9653)
tensor(0.9655)
=================================2=================================
acc : 0.9769375324249268, f1 : 0.9767911434173584
=================================test score=================================
tensor(0.9669)
tensor(0.9668)
=================================3=================================
acc : 0.9775460958480835, f1 : 0.9777652025222778
=================================test score=================================
tensor(0.9646)
tensor(0.9649)
=================================4=================================
acc : 0.9816775321960449, f1 : 0.

In [134]:
model2=torch.load("./model.pth")
model2.eval()


output = model2(x_test)
print(f"=================================test score=================================")
print(accuracy(output, y_test, task="multiclass", num_classes=10, average="macro"))
print(f1_score(output, y_test, task="multiclass", num_classes=10, average="macro"))

=================================test score=================================
tensor(0.9774)
tensor(0.9775)
